In [371]:
import cv2
import pandas as pd
import os
import numpy as np

In [372]:
def calcular_centro_grilla(x_, y_, pixeles_por_mm, num_filas):
    size_celda = int(pixeles_por_mm)
    centro_x = x_
    centro_y = y_ - (num_filas * size_celda)
    return centro_x, centro_y, size_celda

In [373]:
def dibujar_grilla(img, centro_x, centro_y, y_inferior, tamanio_celda, num_columnas, num_filas):
    for i in range(num_columnas + 1):
        x = centro_x - (num_columnas // 2) * tamanio_celda + i * tamanio_celda
        cv2.line(img, (x, y_inferior), (x, centro_y), (0, 255, 0), 1)

    for i in range(num_filas + 1):
        y = y_inferior - i * tamanio_celda
        cv2.line(img, (centro_x - (num_columnas // 2) * tamanio_celda, y),
                 (centro_x + (num_columnas // 2) * tamanio_celda, y), (0, 255, 0), 1)

In [374]:
def obtener_posicion_grilla(punto_x, punto_y, centro_x, centro_y, tamanio_celda, num_columnas, num_filas):
    # Calcular la coordenada x del borde izquierdo de la cuadrícula
    borde_izquierdo_x = centro_x - (num_columnas // 2) * tamanio_celda
    # Calcular la coordenada y del borde inferior de la cuadrícula
    borde_inferior_y = centro_y + num_filas * tamanio_celda

    # Calcular la distancia horizontal desde el borde izquierdo hasta el punto seleccionado
    distancia_x = punto_x - borde_izquierdo_x
    # Calcular la distancia vertical desde el borde inferior hasta el punto seleccionado
    distancia_y = borde_inferior_y - punto_y

    # Calcular el índice de la columna (grilla_x) y de la fila (grilla_y)
    grilla_x = distancia_x // tamanio_celda
    grilla_y = distancia_y // tamanio_celda

    return int(grilla_x), int(grilla_y)

In [375]:
def crear_matriz(numero_de_columnas, numero_de_filas, columna_marcada, fila_marcada):

    # Crear una matriz de ceros con el tamaño especificado
    matriz = np.zeros((numero_de_filas, numero_de_columnas), dtype=int)

    # Verificar que los índices de la cuadrícula estén dentro de los límites
    if indices_validos(columna_marcada, fila_marcada, numero_de_columnas, numero_de_filas):
        # Marcar la posición correspondiente con un 1
        matriz[fila_marcada, columna_marcada] = 1

    return matriz

def indices_validos(columna, fila, numero_de_columnas, numero_de_filas):
    """
    Verifica si los índices de la cuadrícula están dentro de los límites permitidos.
    """
    columna_valida = 0 <= columna < numero_de_columnas
    fila_valida = 0 <= fila < numero_de_filas
    return columna_valida and fila_valida

In [376]:
def dibujar_puntos(imagen, punto_referencia, punto_marcado):
    cv2.circle(imagen, punto_referencia, 5, (0, 0, 255), -1)  # Punto de referencia en rojo
    cv2.circle(imagen, punto_marcado, 5, (255, 0, 0), -1)


In [377]:
def procesar_imagen_individual2(ruta_imagen, punto_referencia, punto_marcado, pixeles_por_3mm, directorio_salida):
    if not os.path.exists(ruta_imagen):
        print(f"El archivo de imagen no existe: {ruta_imagen}")
        return

    imagen = cv2.imread(ruta_imagen)
    if imagen is None:
        print(f"No se pudo cargar la imagen: {ruta_imagen}")
        return

    pixeles_por_mm = pixeles_por_3mm 
    num_columnas, num_filas = 8, 6

    dibujar_puntos(imagen, punto_referencia, punto_marcado)

    centro_x, centro_y, tamanio_celda = calcular_centro_grilla(punto_referencia[0], punto_referencia[1], pixeles_por_mm, num_filas)
    dibujar_grilla(imagen, centro_x, centro_y, punto_referencia[1], tamanio_celda, num_columnas, num_filas)

    grilla_x, grilla_y = obtener_posicion_grilla(punto_marcado[0], punto_marcado[1], centro_x, centro_y, tamanio_celda, num_columnas, num_filas)
    matriz_grilla = crear_matriz(num_columnas, num_filas, grilla_x, grilla_y)

    #print(f"El punto marcado está en la cuadrícula: ({grilla_x}, {grilla_y})")
    #print("Matriz de la grilla:")
    #print(matriz_grilla)

    ruta_salida = os.path.join(directorio_salida, os.path.basename(ruta_imagen))
    cv2.imwrite(ruta_salida, imagen)
    print(f"Imagen guardada en: {ruta_salida}")

    return matriz_grilla

In [378]:
def procesar_conjunto_datos2(ruta_conjunto_datos, directorio_salida):
    conjunto_datos = pd.read_csv(ruta_conjunto_datos)
    os.makedirs(directorio_salida, exist_ok=True)

    for indice, fila in conjunto_datos.iterrows():
        nombre_imagen = fila["imagen"]
        ruta_imagen = os.path.join('..', 'imagenes', 'seleccionadas', 'medio', nombre_imagen)

        punto_referencia = (int(fila['punto_referencia_x']), int(fila['punto_referencia_y']))
        mm_por_pixel = fila['mm_por_pixel']

        # Aquí definimos manualmente el punto marcado para cada imagen
        punto_marcado = (punto_referencia[0] + 50, punto_referencia[1] - 50)  # Ejemplo: 50 píxeles a la derecha y 50 arriba del punto de referencia

        matriz_grilla = procesar_imagen_individual2(ruta_imagen, punto_referencia, punto_marcado, mm_por_pixel, directorio_salida)

        # Aquí puedes hacer algo con matriz_grilla si lo necesitas, como guardarlo en un archivo

    print("\nProcesamiento completado.")
    print(f"Directorio de salida: {os.path.abspath(directorio_salida)}")

In [379]:
def procesar_imagen_individual(ruta_imagen, punto_referencia, puntos_marcados, pixeles_por_3mm, directorio_salida):
    if not os.path.exists(ruta_imagen):
        print(f"El archivo de imagen no existe: {ruta_imagen}")
        return

    imagen = cv2.imread(ruta_imagen)
    if imagen is None:
        print(f"No se pudo cargar la imagen: {ruta_imagen}")
        return

    pixeles_por_mm = pixeles_por_3mm
    num_columnas, num_filas = 8, 5

    dibujar_puntos(imagen, punto_referencia, punto_referencia)  # Dibujar el punto de referencia

    centro_x, centro_y, tamanio_celda = calcular_centro_grilla(punto_referencia[0], punto_referencia[1], pixeles_por_mm, num_filas)
    dibujar_grilla(imagen, centro_x, centro_y, punto_referencia[1], tamanio_celda, num_columnas, num_filas)

    matriz_grilla = crear_matriz(num_columnas, num_filas, -1, -1)  # Inicializar la matriz sin puntos marcados

    for punto_marcado in puntos_marcados:
        dibujar_puntos(imagen, punto_referencia, punto_marcado)
        grilla_x, grilla_y = obtener_posicion_grilla(punto_marcado[0], punto_marcado[1], centro_x, centro_y, tamanio_celda, num_columnas, num_filas)
        matriz_grilla[grilla_y][grilla_x] = 1  # Marcar el punto en la matriz

    # Asegurarse de que directorio_salida es un directorio
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)

    ruta_salida = os.path.join(directorio_salida, os.path.basename(ruta_imagen))
    cv2.imwrite(ruta_salida, imagen)
    print(f"Imagen guardada en: {ruta_salida}")

    return matriz_grilla

In [380]:
def procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida):
    conjunto_datos = pd.read_csv(ruta_conjunto_datos)
    os.makedirs(directorio_salida, exist_ok=True)

    for indice, fila in conjunto_datos.iterrows():
        nombre_imagen = fila["imagen"]
        ruta_imagen = os.path.join('..', 'imagenes', 'seleccionadas', 'medio', nombre_imagen)

        punto_referencia = (int(fila['punto_referencia_x']), int(fila['punto_referencia_y']))
        mm_por_pixel = fila['mm_por_pixel']

        if not os.path.exists(ruta_imagen):
            print(f"ERROR: El archivo de imagen no existe: {ruta_imagen}")
            print("Comprueba que la ruta sea correcta y que el archivo esté presente.")
            continue

        puntos_marcados = []
        for i in range(1, 16):
            coord_x = f'coordenada_x_punto {i}'
            coord_y = f'coordenada_y_punto {i}'

            if coord_x in fila and coord_y in fila:
                punto_marcado = (int(fila[coord_x]), int(fila[coord_y]))
                puntos_marcados.append(punto_marcado)
            else:
                print(f"No se encontraron coordenadas para el punto {i}")

        if puntos_marcados:
            matriz_grilla = procesar_imagen_individual(ruta_imagen, punto_referencia, puntos_marcados, mm_por_pixel, directorio_salida)
            # Aquí puedes hacer algo con matriz_grilla si lo necesitas, como guardarlo en un archivo

    print("\nProcesamiento completado.")
    print(f"Directorio de salida: {os.path.abspath(directorio_salida)}")



In [381]:
if __name__ == "__main__":
    ruta_conjunto_datos = '../datos/df.csv'
    directorio_salida = '../imagenes/procesadas/'
    procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida)

ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon11_Seccion 1 horizontal C11.jpg
Comprueba que la ruta sea correcta y que el archivo esté presente.
ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon12_Seccion 1 horizontal C12.jpg
Comprueba que la ruta sea correcta y que el archivo esté presente.
ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon17_Seccion 1 horizontal C17.jpg
Comprueba que la ruta sea correcta y que el archivo esté presente.
ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon8_Seccion 1 horizontal C8.jpg
Comprueba que la ruta sea correcta y que el archivo esté presente.
ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon3_Seccion 1 horizontal C3.jpg
Comprueba que la ruta sea correcta y que el archivo esté presente.
ERROR: El archivo de imagen no existe: ../imagenes/seleccionadas/medio/Cordon19_Seccion 1 horizontal C19.jpg
Comprueba que l

IndexError: index -10 is out of bounds for axis 0 with size 6